# Planners

## Global and Local Planners


In robotics, we generally have two planners namely, Global and Local Planners. While global planners produce a rough path from start point to goal point, local planners run much more frequently to avoid local obstacles along the path produced by the global planners. These planners are utilized in many different kinds of robotics applications from autonomous cars to robotics arms.

## Planner Types

Planners can be categorized according to the way they work. While some planners produce plans searching through a a grid structure, others randomly sample the search space to reach at the goal. The former planners are called *grid search-based planners*. The latter ones are known as *sampling-based planners.* 

Because grid search-based planners requires a discritized grid cells, they tend to demand more computational power. Sampling-based planners on the other hand create a plan by selecting random points in the free configuration space without any discritization.


Example grid search based planners are Breath First Search, Dijkstra, A*...

Example sampling-based planners are RRT(Rapidly Exploring Random Tree), RRTStar, RRTConnect, PRM(Probabilistic Road Map).

## RRT

RRT is a sampling-based planning algorithm.

Here is a single step of RRT algorithm:

```python
    def step(self):
        # 1 - Get a random node, occasionally select the goal in place of
        # random node.
        random_node = self.get_random_node()
        # 2 - Find existing nearest node to the random node
        nearest_ind = self.get_nearest_node_index(self.node_list, random_node)
        nearest_node = self.node_list[nearest_ind]
        # 3 - Expand towards the random node from the nearest node with max of
        # given extand distance
        new_node = self.steer(nearest_node, random_node, self.expand_dis)

        # 4 - If no collision add the new expand node to the list.
        if self.check_collision(new_node, self.obstacle_list):
            self.node_list.append(new_node)

        # 5- If the goal is within the expand distance from the last node,
        # directly expand towards the goal node. If no collision, the path is
        # found.
        if self.calc_dist_to_goal(self.node_list[-1].x,
                                    self.node_list[-1].y) <= self.expand_dis:
            final_node = self.steer(self.node_list[-1], self.end,
                                    self.expand_dis)
            if self.check_collision(final_node, self.obstacle_list):
                self.path = self.generate_final_course(len(self.node_list) - 1)
```


![RRT](images/rrt.png "RRT")

- Path is low quality, possibly with a lot of sharp points.
- Path won't systematically improve over the iteration.
- Doesn't guarantee any optimal path.

## RRTStar

RRTStar is an extension to the RRT. It creates nodes the same way RRT does.
The difference comes from the way it connects the nodes to the tree.

```python
    def step(self):
        # 1 - Get a random node, occasionally select the goal in place of
        # random node.
        random_node = self.get_random_node()
        # 2 - Find existing nearest node to the random node
        nearest_ind = self.get_nearest_node_index(self.node_list, random_node)
        nearest_node = self.node_list[nearest_ind]

        # 3 - Find the node that expands towards the random node from the
        # nearest node with max of given extand distance
        new_node = self.steer(self.node_list[nearest_ind], random_node,
                                self.expand_dis)

        # 4 - Compute the cost to reach at this new node from the starting node
        new_node.cost = nearest_node.cost + \
            math.hypot(new_node.x-nearest_node.x,
                        new_node.y-nearest_node.y)

        # 5 - If no collision, we don't connect the new node to the nearest of
        # the random node, but we select a nearest node within a circle
        # centered at new node with radius expand distance or we apply some
        # dynamic radius such that the circle gets smaller as we add nodes.
        # In other words, new node changes its parent.

        # 6- We also rewire the nodes within the circle if the new node can
        # reduce their costs for coming from the starting point.
        if self.check_collision(new_node, self.obstacle_list):
            near_inds = self.find_near_nodes(new_node)
            node_with_updated_parent = self.choose_parent(
                new_node, near_inds)
            if node_with_updated_parent:
                self.rewire(node_with_updated_parent, near_inds)
                self.node_list.append(node_with_updated_parent)
            else:
                self.node_list.append(new_node)

        # 7- Check if there is a path from a node to the goal
        # without a collision.
        last_index = self.search_best_goal_node()
        if last_index is not None:
            self.path = self.generate_final_course(last_index)
```

![RRT*](images/rrtstar.png "RRT*")

- Path can approach to optimal over the iterations (rewiring).
- Could produce smoother paths compared to RRT.
- Tree produces a fan-like structure.

## Breadth First Search

Grid based search. Can be implemented using a Open Queue and Closed Set/Dict.

```
open = Queue()
open.put(start )
closed = Dict()
closed[start] = None

while not frontier.empty():
   current = open.get()
   for next in graph.neighbors(current):
      if next not in closed:
         open.put(next)
         closed[next] = current
```

![Bread First Search](images/breadth-first-search.png "Breadth First Search")

- Right, Left, Top have cost 1, Bottom has cost 2.
- Note the grids.
- Optimal path

## Dijkstra

Similar to Breadth First Search, but this time we also account for the cost reaching to a node from the start node. We prioritize the nodes to visit first according to the low cost values. We can use a Priority Queue for this algorithm. 

![Dijkstra](images/dijkstra.png "Dijkstra")

- Because going to bottom is more costly, this algorithm avoids it. But it still visits many nodes in the grid search space.
- Optimal path


## A*

In addition to the cost for coming from the starting node, A* also introduces a heuristic to go to the goal node from the current node. The heuristic should be admissible. Famous heuristics are Euclidan distance, Menhatten Distance, Hamming Distance. We can again use Priority Queue, but this time priority is computed as cost plus the heuristic.

![A*](images/astar.png "A*")

- Visits fewer nodes. More targetted torwards to goal.
- Optimal path


## PRM

Randomly sample the free space. Connect samples together with some rule(k-nearest, circle with some radius etc..) Then run any search based algorithm over the samples.

![PRM](images/prm.png "PRM")

- Path is not optimal
- Once sampled, multiple query search is an advantage over RRT family.
- Preferred for higher dimensional search spaces.
- Deals with fewer nodes compared to grid search-based algorithms.

Robotic arm PRM example:

![PRM Robotic Arm](images/prm2.png "PRM Robotic Arm")

Full code can be found [here](https://github.com/gokhanettin/robotics-stuff/blob/master/ceng782/assignment3.ipynb).


# OMPL RRT* Demo

Obstacle count, obstacle radius, square area side length, and run time for `solve` method  is provided through `rosparam`.

| Experiment No | Obstacle Count | Obstacle Radius | Square Area Side Length | Run Time |PathLengthOptimizationObjective Threshold | Found | TimeElapsed |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | 30 | 5 meters | 100 meters | 1.0 seconds | N/A | True | 1000.15473366 miliseconds |
| 2 | 30 | 5 meters | 100 meters | 1.0 seconds | 1000 | True | 4.7869682312 miliseconds |

## Experiment 1

Launch rrtstar and rviz:

```sh
roslaunch rrtstar rrtstar.launch
```

Call the planner with arguments startX, startY, goalX, and goalY arguments.

```sh
rosservice call /make_plan 0 0 100 100
found: True
elapsedTimeMiliseconds: 1000.15473366
```
![Experiment 1](images/experiment1.png "Experiment 1")

## Experiment 2

Launch rrtstar and rviz:

```sh
roslaunch rrtstar rrtstar.launch
```

Call the planner with arguments startX, startY, goalX, and goalY arguments.

```sh
rosservice call /make_plan 0 0 100 100
found: True
elapsedTimeMiliseconds: 4.7869682312
```
![Experiment 2](images/experiment2.png "Experiment 2")